mb syn

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import hamming_loss, precision_score, recall_score, f1_score
from PIL import Image
import cv2

# -------------------- Config --------------------
test_dir = ""   # <-- your test folder
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_names = ["left", "right", "forward"]
num_classes = len(class_names)

mobilenet_weights = ""
resnet_weights    = ""



# -------------------- Dataset --------------------
class MultiLabelDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        for subfolder in os.listdir(root_dir):
            sub_path = os.path.join(root_dir, subfolder)
            if not os.path.isdir(sub_path):
                continue
            label_vector = [int(x) for x in subfolder.split('_')]
            for fname in os.listdir(sub_path):
                if fname.lower().endswith(('.png','.jpg','.jpeg')):
                    self.images.append(os.path.join(sub_path,fname))
                    self.labels.append(label_vector)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        img = Image.open(img_path).convert("L")  # grayscale
        if self.transform:
            img = self.transform(img)

        # expand grayscale → 3 channels
        img = img.repeat(3, 1, 1)
        
        return img, label

# -------------------- Transforms --------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_dataset = MultiLabelDataset(test_dir, transform=transform)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# -------------------- Model Builder --------------------
def build_model(arch="mobilenetv2", weight_path=None, pretrained=True):
    if arch == "mobilenetv2":
        model = models.mobilenet_v2(pretrained=pretrained)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif arch == "resnet50":
        model = models.resnet50(pretrained=pretrained)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        raise ValueError("Unsupported arch")

    if weight_path:
        model.load_state_dict(torch.load(weight_path, map_location=device))

    return model.to(device)


# -------------------- Metrics --------------------
def evaluate_model(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.cpu().numpy())

    all_preds  = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    print("\n===== Evaluation =====")
    print(f"Hamming Loss : {hamming_loss(all_labels, all_preds):.4f}")
    print(f"Precision    : {precision_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"Recall       : {recall_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"F1 Score     : {f1_score(all_labels, all_preds, average='micro'):.4f}")


# -------------------- Draw & Save Predictions --------------------
def save_predictions_with_overlay(model, loader, arch_name, save_root="test_predictions", threshold=0.5):
    model.eval()
    save_dir = os.path.join(save_root, arch_name)
    os.makedirs(save_dir, exist_ok=True)

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images = images.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            labels = labels.cpu().numpy()

            for i in range(len(images)):
                pred_vec = preds[i]
                label_vec = labels[i]

                pred_classes = [class_names[j] for j, v in enumerate(pred_vec) if v == 1]
                true_classes = [class_names[j] for j, v in enumerate(label_vec) if v == 1]

                pred_text = "Pred: " + (",".join(pred_classes) if pred_classes else "None")
                true_text = "True: " + (",".join(true_classes) if true_classes else "None")

                img_path = loader.dataset.images[batch_idx * loader.batch_size + i]
                img = cv2.imread(img_path)
                if img is None:
                    continue

                img = cv2.resize(img, (400, 400))
                cv2.putText(img, pred_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                cv2.putText(img, true_text, (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

                save_path = os.path.join(save_dir, f"overlay_{os.path.basename(img_path)}")
                cv2.imwrite(save_path, img)

    print(f"Overlay predictions saved to: {save_dir}")



# Pretrained=False
mobilenet_nf = build_model("mobilenetv2", mobilenet_weights, pretrained=False)
evaluate_model(mobilenet_nf, test_loader)



/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



===== Evaluation =====
Hamming Loss : 0.2367
Precision    : 0.6862
Recall       : 0.9149
F1 Score     : 0.7842


rn syn

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import hamming_loss, precision_score, recall_score, f1_score
from PIL import Image
import cv2

# -------------------- Config --------------------
test_dir = ""   # <-- your test folder
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_names = ["left", "right", "forward"]
num_classes = len(class_names)

mobilenet_weights = ""
resnet_weights    = ""


# -------------------- Dataset --------------------
class MultiLabelDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images, self.labels = [], []

        for subfolder in os.listdir(root_dir):
            sub_path = os.path.join(root_dir, subfolder)
            if not os.path.isdir(sub_path):
                continue
            label_vec = [int(x) for x in subfolder.split("_")]
            for fname in os.listdir(sub_path):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.images.append(os.path.join(sub_path, fname))
                    self.labels.append(label_vec)
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        img = Image.open(img_path).convert("L")  # grayscale
        if self.transform:
            img = self.transform(img)

        # expand grayscale → 3 channels
        img = img.repeat(3, 1, 1)
        
        return img, label

# -------------------- Transforms --------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_dataset = MultiLabelDataset(test_dir, transform=transform)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# -------------------- Model Builder --------------------
def build_model(arch="resnet50", weight_path=None, pretrained=True):
    if arch == "mobilenetv2":
        model = models.mobilenet_v2(pretrained=pretrained)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif arch == "resnet50":
        model = models.resnet50(pretrained=pretrained)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        raise ValueError("Unsupported arch")

    if weight_path:
        model.load_state_dict(torch.load(weight_path, map_location=device))

    return model.to(device)


# -------------------- Metrics --------------------
def evaluate_model(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.cpu().numpy())

    all_preds  = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    print("\n===== Evaluation =====")
    print(f"Hamming Loss : {hamming_loss(all_labels, all_preds):.4f}")
    print(f"Precision    : {precision_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"Recall       : {recall_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"F1 Score     : {f1_score(all_labels, all_preds, average='micro'):.4f}")


# -------------------- Draw & Save Predictions --------------------
def save_predictions_with_overlay(model, loader, arch_name, save_root="test_predictions", threshold=0.5):
    model.eval()
    save_dir = os.path.join(save_root, arch_name)
    os.makedirs(save_dir, exist_ok=True)

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images = images.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            labels = labels.cpu().numpy()

            for i in range(len(images)):
                pred_vec = preds[i]
                label_vec = labels[i]

                pred_classes = [class_names[j] for j, v in enumerate(pred_vec) if v == 1]
                true_classes = [class_names[j] for j, v in enumerate(label_vec) if v == 1]

                pred_text = "Pred: " + (",".join(pred_classes) if pred_classes else "None")
                true_text = "True: " + (",".join(true_classes) if true_classes else "None")

                img_path = loader.dataset.images[batch_idx * loader.batch_size + i]
                img = cv2.imread(img_path)
                if img is None:
                    continue

                img = cv2.resize(img, (400, 400))
                cv2.putText(img, pred_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                cv2.putText(img, true_text, (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

                save_path = os.path.join(save_dir, f"overlay_{os.path.basename(img_path)}")
                cv2.imwrite(save_path, img)

    print(f"Overlay predictions saved to: {save_dir}")



# Pretrained=False
resnet_nf = build_model("resnet50", resnet_weights, pretrained=False)
evaluate_model(resnet_nf, test_loader)



/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



===== Evaluation =====
Hamming Loss : 0.2500
Precision    : 0.6667
Recall       : 0.9362
F1 Score     : 0.7788


mb_img

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import hamming_loss, precision_score, recall_score, f1_score
from PIL import Image
import cv2

# -------------------- Config --------------------
test_dir = ""   # <-- your test folder
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_names = ["left", "right", "forward"]
num_classes = len(class_names)

mobilenet_weights = ""
resnet_weights    = ""


# -------------------- Dataset --------------------
class MultiLabelDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images, self.labels = [], []

        for subfolder in os.listdir(root_dir):
            sub_path = os.path.join(root_dir, subfolder)
            if not os.path.isdir(sub_path):
                continue
            label_vec = [int(x) for x in subfolder.split("_")]
            for fname in os.listdir(sub_path):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.images.append(os.path.join(sub_path, fname))
                    self.labels.append(label_vec)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        img = Image.open(img_path).convert("L")  # grayscale
        if self.transform:
            img = self.transform(img)

        # expand grayscale → 3 channels
        img = img.repeat(3, 1, 1)
        
        return img, label


# -------------------- Transforms --------------------
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_dataset = MultiLabelDataset(test_dir, transform=transform)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# -------------------- Model Builder --------------------
def build_model(arch="mobilenetv2", weight_path=None, pretrained=True):
    if arch == "mobilenetv2":
        model = models.mobilenet_v2(pretrained=pretrained)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif arch == "resnet50":
        model = models.resnet50(pretrained=pretrained)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        raise ValueError("Unsupported arch")

    if weight_path:
        model.load_state_dict(torch.load(weight_path, map_location=device))

    return model.to(device)


# -------------------- Metrics --------------------
def evaluate_model(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.cpu().numpy())

    all_preds  = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    print("\n===== Evaluation =====")
    print(f"Hamming Loss : {hamming_loss(all_labels, all_preds):.4f}")
    print(f"Precision    : {precision_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"Recall       : {recall_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"F1 Score     : {f1_score(all_labels, all_preds, average='micro'):.4f}")


# -------------------- Draw & Save Predictions --------------------
def save_predictions_with_overlay(model, loader, arch_name, save_root="test_predictions", threshold=0.5):
    model.eval()
    save_dir = os.path.join(save_root, arch_name)
    os.makedirs(save_dir, exist_ok=True)

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images = images.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            labels = labels.cpu().numpy()

            for i in range(len(images)):
                pred_vec = preds[i]
                label_vec = labels[i]

                pred_classes = [class_names[j] for j, v in enumerate(pred_vec) if v == 1]
                true_classes = [class_names[j] for j, v in enumerate(label_vec) if v == 1]

                pred_text = "Pred: " + (",".join(pred_classes) if pred_classes else "None")
                true_text = "True: " + (",".join(true_classes) if true_classes else "None")

                img_path = loader.dataset.images[batch_idx * loader.batch_size + i]
                img = cv2.imread(img_path)
                if img is None:
                    continue

                img = cv2.resize(img, (400, 400))
                cv2.putText(img, pred_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                cv2.putText(img, true_text, (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

                save_path = os.path.join(save_dir, f"overlay_{os.path.basename(img_path)}")
                cv2.imwrite(save_path, img)

    print(f"Overlay predictions saved to: {save_dir}")


# Pretrained=False
mobilenet_nf = build_model("mobilenetv2", mobilenet_weights, pretrained=False)
evaluate_model(mobilenet_nf, test_loader)


/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



===== Evaluation =====
Hamming Loss : 0.2233
Precision    : 0.6968
Recall       : 0.9291
F1 Score     : 0.7964


rn_img

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import hamming_loss, precision_score, recall_score, f1_score
from PIL import Image
import cv2

# -------------------- Config --------------------
test_dir = ""   # <-- your test folder
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_names = ["left", "right", "forward"]
num_classes = len(class_names)

mobilenet_weights = ""
resnet_weights    = ""


# -------------------- Dataset --------------------
class MultiLabelDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        for subfolder in os.listdir(root_dir):
            sub_path = os.path.join(root_dir, subfolder)
            if not os.path.isdir(sub_path):
                continue
            label_vector = [int(x) for x in subfolder.split('_')]
            for fname in os.listdir(sub_path):
                if fname.lower().endswith(('.png','.jpg','.jpeg')):
                    self.images.append(os.path.join(sub_path,fname))
                    self.labels.append(label_vector)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        img = Image.open(img_path).convert("L")  # grayscale
        if self.transform:
            img = self.transform(img)

        # expand grayscale → 3 channels
        img = img.repeat(3, 1, 1)
        
        return img, label

# -------------------- Transforms --------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_dataset = MultiLabelDataset(test_dir, transform=transform)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# -------------------- Model Builder --------------------
def build_model(arch="resnet50", weight_path=None, pretrained=True):
    if arch == "mobilenetv2":
        model = models.mobilenet_v2(pretrained=pretrained)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif arch == "resnet50":
        model = models.resnet50(pretrained=pretrained)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        raise ValueError("Unsupported arch")

    if weight_path:
        model.load_state_dict(torch.load(weight_path, map_location=device))

    return model.to(device)


# -------------------- Metrics --------------------
def evaluate_model(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.cpu().numpy())

    all_preds  = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    print("\n===== Evaluation =====")
    print(f"Hamming Loss : {hamming_loss(all_labels, all_preds):.4f}")
    print(f"Precision    : {precision_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"Recall       : {recall_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"F1 Score     : {f1_score(all_labels, all_preds, average='micro'):.4f}")


# -------------------- Draw & Save Predictions --------------------
def save_predictions_with_overlay(model, loader, arch_name, save_root="test_predictions", threshold=0.5):
    model.eval()
    save_dir = os.path.join(save_root, arch_name)
    os.makedirs(save_dir, exist_ok=True)

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images = images.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            labels = labels.cpu().numpy()

            for i in range(len(images)):
                pred_vec = preds[i]
                label_vec = labels[i]

                pred_classes = [class_names[j] for j, v in enumerate(pred_vec) if v == 1]
                true_classes = [class_names[j] for j, v in enumerate(label_vec) if v == 1]

                pred_text = "Pred: " + (",".join(pred_classes) if pred_classes else "None")
                true_text = "True: " + (",".join(true_classes) if true_classes else "None")

                img_path = loader.dataset.images[batch_idx * loader.batch_size + i]
                img = cv2.imread(img_path)
                if img is None:
                    continue

                img = cv2.resize(img, (400, 400))
                cv2.putText(img, pred_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                cv2.putText(img, true_text, (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

                save_path = os.path.join(save_dir, f"overlay_{os.path.basename(img_path)}")
                cv2.imwrite(save_path, img)

    print(f"Overlay predictions saved to: {save_dir}")



# Pretrained=False
resnet_nf = build_model("resnet50", resnet_weights, pretrained=False)
evaluate_model(resnet_nf, test_loader)



/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



===== Evaluation =====
Hamming Loss : 0.0767
Precision    : 0.8831
Recall       : 0.9645
F1 Score     : 0.9220


mb_All_lay

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import hamming_loss, precision_score, recall_score, f1_score
from PIL import Image
import cv2

# -------------------- Config --------------------
test_dir = ""   # <-- your test folder
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_names = ["left", "right", "forward"]
num_classes = len(class_names)

mobilenet_weights = ""
resnet_weights    = ""


# -------------------- Dataset --------------------
class MultiLabelDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        for subfolder in os.listdir(root_dir):
            sub_path = os.path.join(root_dir, subfolder)
            if not os.path.isdir(sub_path):
                continue
            label_vector = [int(x) for x in subfolder.split('_')]
            for fname in os.listdir(sub_path):
                if fname.lower().endswith(('.png','.jpg','.jpeg')):
                    self.images.append(os.path.join(sub_path,fname))
                    self.labels.append(label_vector)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        img = Image.open(img_path).convert("L")  # grayscale
        if self.transform:
            img = self.transform(img)

        # expand grayscale → 3 channels
        img = img.repeat(3, 1, 1)
        
        return img, label

# -------------------- Transforms --------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_dataset = MultiLabelDataset(test_dir, transform=transform)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# -------------------- Model Builder --------------------
def build_model(arch="mobilenetv2", weight_path=None, pretrained=True):
    if arch == "mobilenetv2":
        model = models.mobilenet_v2(pretrained=pretrained)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif arch == "resnet50":
        model = models.resnet50(pretrained=pretrained)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        raise ValueError("Unsupported arch")

    if weight_path:
        model.load_state_dict(torch.load(weight_path, map_location=device))

    return model.to(device)


# -------------------- Metrics --------------------
def evaluate_model(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.cpu().numpy())

    all_preds  = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    print("\n===== Evaluation =====")
    print(f"Hamming Loss : {hamming_loss(all_labels, all_preds):.4f}")
    print(f"Precision    : {precision_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"Recall       : {recall_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"F1 Score     : {f1_score(all_labels, all_preds, average='micro'):.4f}")


# -------------------- Draw & Save Predictions --------------------
def save_predictions_with_overlay(model, loader, arch_name, save_root="test_predictions", threshold=0.5):
    model.eval()
    save_dir = os.path.join(save_root, arch_name)
    os.makedirs(save_dir, exist_ok=True)

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images = images.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            labels = labels.cpu().numpy()

            for i in range(len(images)):
                pred_vec = preds[i]
                label_vec = labels[i]

                pred_classes = [class_names[j] for j, v in enumerate(pred_vec) if v == 1]
                true_classes = [class_names[j] for j, v in enumerate(label_vec) if v == 1]

                pred_text = "Pred: " + (",".join(pred_classes) if pred_classes else "None")
                true_text = "True: " + (",".join(true_classes) if true_classes else "None")

                img_path = loader.dataset.images[batch_idx * loader.batch_size + i]
                img = cv2.imread(img_path)
                if img is None:
                    continue

                img = cv2.resize(img, (400, 400))
                cv2.putText(img, pred_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                cv2.putText(img, true_text, (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

                save_path = os.path.join(save_dir, f"overlay_{os.path.basename(img_path)}")
                cv2.imwrite(save_path, img)

    print(f"Overlay predictions saved to: {save_dir}")



# Pretrained=False
mobilenet_nf = build_model("mobilenetv2", mobilenet_weights, pretrained=False)
evaluate_model(mobilenet_nf, test_loader)



/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



===== Evaluation =====
Hamming Loss : 0.2367
Precision    : 0.6733
Recall       : 0.9645
F1 Score     : 0.7930


r_net_all_lay

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import hamming_loss, precision_score, recall_score, f1_score
from PIL import Image
import cv2

# -------------------- Config --------------------
test_dir = ""   # <-- your test folder
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_names = ["left", "right", "forward"]
num_classes = len(class_names)

mobilenet_weights = ""
resnet_weights    = ""


# -------------------- Dataset --------------------
class MultiLabelDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images, self.labels = [], []

        for subfolder in os.listdir(root_dir):
            sub_path = os.path.join(root_dir, subfolder)
            if not os.path.isdir(sub_path):
                continue
            label_vec = [int(x) for x in subfolder.split("_")]
            for fname in os.listdir(sub_path):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.images.append(os.path.join(sub_path, fname))
                    self.labels.append(label_vec)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        img = Image.open(img_path).convert("L")  # grayscale
        if self.transform:
            img = self.transform(img)

        # expand grayscale → 3 channels
        img = img.repeat(3, 1, 1)
        
        return img, label

# -------------------- Transforms --------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_dataset = MultiLabelDataset(test_dir, transform=transform)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# -------------------- Model Builder --------------------
def build_model(arch="resnet50", weight_path=None, pretrained=True):
    if arch == "mobilenetv2":
        model = models.mobilenet_v2(pretrained=pretrained)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif arch == "resnet50":
        model = models.resnet50(pretrained=pretrained)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        raise ValueError("Unsupported arch")

    if weight_path:
        model.load_state_dict(torch.load(weight_path, map_location=device))

    return model.to(device)


# -------------------- Metrics --------------------
def evaluate_model(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.cpu().numpy())

    all_preds  = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    print("\n===== Evaluation =====")
    print(f"Hamming Loss : {hamming_loss(all_labels, all_preds):.4f}")
    print(f"Precision    : {precision_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"Recall       : {recall_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"F1 Score     : {f1_score(all_labels, all_preds, average='micro'):.4f}")


# -------------------- Draw & Save Predictions --------------------
def save_predictions_with_overlay(model, loader, arch_name, save_root="test_predictions", threshold=0.5):
    model.eval()
    save_dir = os.path.join(save_root, arch_name)
    os.makedirs(save_dir, exist_ok=True)

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images = images.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            labels = labels.cpu().numpy()

            for i in range(len(images)):
                pred_vec = preds[i]
                label_vec = labels[i]

                pred_classes = [class_names[j] for j, v in enumerate(pred_vec) if v == 1]
                true_classes = [class_names[j] for j, v in enumerate(label_vec) if v == 1]

                pred_text = "Pred: " + (",".join(pred_classes) if pred_classes else "None")
                true_text = "True: " + (",".join(true_classes) if true_classes else "None")

                img_path = loader.dataset.images[batch_idx * loader.batch_size + i]
                img = cv2.imread(img_path)
                if img is None:
                    continue

                img = cv2.resize(img, (400, 400))
                cv2.putText(img, pred_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                cv2.putText(img, true_text, (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

                save_path = os.path.join(save_dir, f"overlay_{os.path.basename(img_path)}")
                cv2.imwrite(save_path, img)

    print(f"Overlay predictions saved to: {save_dir}")


# Pretrained=False
resnet_nf = build_model("resnet50", resnet_weights, pretrained=False)
evaluate_model(resnet_nf, test_loader)



/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



===== Evaluation =====
Hamming Loss : 0.2067
Precision    : 0.7484
Recall       : 0.8440
F1 Score     : 0.7933


rn he_inint

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import hamming_loss, precision_score, recall_score, f1_score
from PIL import Image
import cv2

# -------------------- Config --------------------
test_dir = ""   # <-- your test folder
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_names = ["left", "right", "forward"]
num_classes = len(class_names)

mobilenet_weights = ""
resnet_weights    = ""


# -------------------- Dataset --------------------
class MultiLabelDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images, self.labels = [], []

        for subfolder in os.listdir(root_dir):
            sub_path = os.path.join(root_dir, subfolder)
            if not os.path.isdir(sub_path):
                continue
            label_vec = [int(x) for x in subfolder.split("_")]
            for fname in os.listdir(sub_path):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.images.append(os.path.join(sub_path, fname))
                    self.labels.append(label_vec)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        img = Image.open(img_path).convert("L")  # grayscale
        if self.transform:
            img = self.transform(img)

        # expand grayscale → 3 channels
        img = img.repeat(3, 1, 1)
        
        return img, label

# -------------------- Transforms --------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_dataset = MultiLabelDataset(test_dir, transform=transform)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# -------------------- Model Builder --------------------
def build_model(arch="resnet50", weight_path=None, pretrained=True):
    if arch == "mobilenetv2":
        model = models.mobilenet_v2(pretrained=pretrained)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif arch == "resnet50":
        model = models.resnet50(pretrained=pretrained)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        raise ValueError("Unsupported arch")

    if weight_path:
        model.load_state_dict(torch.load(weight_path, map_location=device))

    return model.to(device)


# -------------------- Metrics --------------------
def evaluate_model(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.cpu().numpy())

    all_preds  = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    print("\n===== Evaluation =====")
    print(f"Hamming Loss : {hamming_loss(all_labels, all_preds):.4f}")
    print(f"Precision    : {precision_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"Recall       : {recall_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"F1 Score     : {f1_score(all_labels, all_preds, average='micro'):.4f}")


# -------------------- Draw & Save Predictions --------------------
def save_predictions_with_overlay(model, loader, arch_name, save_root="test_predictions", threshold=0.5):
    model.eval()
    save_dir = os.path.join(save_root, arch_name)
    os.makedirs(save_dir, exist_ok=True)

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images = images.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            labels = labels.cpu().numpy()

            for i in range(len(images)):
                pred_vec = preds[i]
                label_vec = labels[i]

                pred_classes = [class_names[j] for j, v in enumerate(pred_vec) if v == 1]
                true_classes = [class_names[j] for j, v in enumerate(label_vec) if v == 1]

                pred_text = "Pred: " + (",".join(pred_classes) if pred_classes else "None")
                true_text = "True: " + (",".join(true_classes) if true_classes else "None")

                img_path = loader.dataset.images[batch_idx * loader.batch_size + i]
                img = cv2.imread(img_path)
                if img is None:
                    continue

                img = cv2.resize(img, (400, 400))
                cv2.putText(img, pred_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                cv2.putText(img, true_text, (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

                save_path = os.path.join(save_dir, f"overlay_{os.path.basename(img_path)}")
                cv2.imwrite(save_path, img)

    print(f"Overlay predictions saved to: {save_dir}")



# Pretrained=False
resnet_nf = build_model("resnet50", resnet_weights, pretrained=False)
evaluate_model(resnet_nf, test_loader)



/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



===== Evaluation =====
Hamming Loss : 0.2200
Precision    : 0.7820
Recall       : 0.7376
F1 Score     : 0.7591


fan_in_He

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import hamming_loss, precision_score, recall_score, f1_score
from PIL import Image
import cv2

# -------------------- Config --------------------
test_dir = ""   # <-- your test folder
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_names = ["left", "right", "forward"]
num_classes = len(class_names)

mobilenet_weights = ""
resnet_weights    = ""


# -------------------- Dataset --------------------
class MultiLabelDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images, self.labels = [], []

        for subfolder in os.listdir(root_dir):
            sub_path = os.path.join(root_dir, subfolder)
            if not os.path.isdir(sub_path):
                continue
            label_vec = [int(x) for x in subfolder.split("_")]
            for fname in os.listdir(sub_path):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.images.append(os.path.join(sub_path, fname))
                    self.labels.append(label_vec)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        img = Image.open(img_path).convert("L")  # grayscale
        if self.transform:
            img = self.transform(img)

        # expand grayscale → 3 channels
        img = img.repeat(3, 1, 1)
        
        return img, label

# -------------------- Transforms --------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_dataset = MultiLabelDataset(test_dir, transform=transform)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# -------------------- Model Builder --------------------
def build_model(arch="resnet50", weight_path=None, pretrained=True):
    if arch == "mobilenetv2":
        model = models.mobilenet_v2(pretrained=pretrained)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif arch == "resnet50":
        model = models.resnet50(pretrained=pretrained)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        raise ValueError("Unsupported arch")

    if weight_path:
        model.load_state_dict(torch.load(weight_path, map_location=device))

    return model.to(device)


# -------------------- Metrics --------------------
def evaluate_model(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.cpu().numpy())

    all_preds  = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    print("\n===== Evaluation =====")
    print(f"Hamming Loss : {hamming_loss(all_labels, all_preds):.4f}")
    print(f"Precision    : {precision_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"Recall       : {recall_score(all_labels, all_preds, average='micro'):.4f}")
    print(f"F1 Score     : {f1_score(all_labels, all_preds, average='micro'):.4f}")


# -------------------- Draw & Save Predictions --------------------
def save_predictions_with_overlay(model, loader, arch_name, save_root="test_predictions", threshold=0.5):
    model.eval()
    save_dir = os.path.join(save_root, arch_name)
    os.makedirs(save_dir, exist_ok=True)

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images = images.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= threshold).int().cpu().numpy()
            labels = labels.cpu().numpy()

            for i in range(len(images)):
                pred_vec = preds[i]
                label_vec = labels[i]

                pred_classes = [class_names[j] for j, v in enumerate(pred_vec) if v == 1]
                true_classes = [class_names[j] for j, v in enumerate(label_vec) if v == 1]

                pred_text = "Pred: " + (",".join(pred_classes) if pred_classes else "None")
                true_text = "True: " + (",".join(true_classes) if true_classes else "None")

                img_path = loader.dataset.images[batch_idx * loader.batch_size + i]
                img = cv2.imread(img_path)
                if img is None:
                    continue

                img = cv2.resize(img, (400, 400))
                cv2.putText(img, pred_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                cv2.putText(img, true_text, (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

                save_path = os.path.join(save_dir, f"overlay_{os.path.basename(img_path)}")
                cv2.imwrite(save_path, img)

    print(f"Overlay predictions saved to: {save_dir}")



# Pretrained=False
resnet_nf = build_model("resnet50", resnet_weights, pretrained=False)
evaluate_model(resnet_nf, test_loader)



/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/intellisense08/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



===== Evaluation =====
Hamming Loss : 0.1933
Precision    : 0.7456
Recall       : 0.8936
F1 Score     : 0.8129
